In [ ]:
import gymnasium as gym
import gym_examples"

import numpy as np
import matplotlib.pyplot as plt

## Goal
The goal of this notebook is to get familiar with the environment. A couple things to note:

- There are 8640 data points in either the `real` or `periodic` dataset. This corresponds to 90 days worth of 15min intervals of LMPs and other data
- You can set the time limit length upon calling. Here, we set it to 2000 to allow for more time.

In [ ]:
nhistory = 10     # number of history points
data = "real" # get periodic (sinusoidal) data - algo "real"
env_mode = "delay" # standard setting but delay cost of buying
start_index = 0
end_index = -1 # 24 * 4
env = gym.make(
    "gym_examples/BatteryEnv-v0", 
    nhistory=nhistory, 
    data=data, 
    mode=env_mode, 
    start_index = start_index,
    end_index = end_index,
    max_episode_steps=2000, # can change length here!
)

Let's take a look at the state
- first index: SOC
- historical prices (length `nhistory`, working backwards in chronological order starting with the price right now)
- current "cost" of electricity (oldest unsold charge)

In [ ]:
print(env.observation_space)

Let's now look at action space.
- action=0 : sell
- action=1 : do nothing
- action=2 : buy

In [ ]:
print(env.action_space)

Let's test out buying, doing nothing, and then selling.

We'll also test out with some "invalid" actions, e.g., selling while empty or buying while full.

In [ ]:
s_0, _ = env.reset()
print(f"Starting state: {s_0}\n")
s_1, _, _, _, _ = env.step(2)
print(f"State after buying: {s_1}\n")
s_2, _, _, _, _ = env.step(1)
print(f"State after doing nothing: {s_2}\n")
s_3, _, _, _, _ = env.step(0)
print(f"State after selling: {s_3}\n")
s_4, _, _, _, _ = env.step(0)
print(f"State after selling with nothing: {s_4}\n")

print("buying a lot to fill up battery...")
for i in range(8):
    s_5, _, _, _, _ = env.step(2)
print(f"State with battery full: {s_5}\n")

s_6, _, _, _, _ = env.step(2)
print(f"State after buying with full battery : {s_6}\n")

Let's visualize the data!

In [ ]:
num_days = 14
num_steps = num_days * 4 * 24

lmp_prices = np.zeros(num_steps)
s_0, _ = env.reset()
lmp_prices[0] = s_0[1]

has_truncated = False
for t in range(1,num_steps):
    s, _, term, trun, _ = env.step(1)
    if trun and not has_truncated:
        print(f"Truncated starting at time {t}")
        has_truncated = True
    lmp_prices[t] = s[1]

ax = plt.subplot()
ax.plot(np.arange(len(lmp_prices)), lmp_prices)
ax.set(
    title=f"{data} LMP prices over {num_days} days", 
    ylabel="Prices ($/MWh)", 
    xlabel="RT time steps"
)

### Modifying time limit

If you want to change the time limit after initializing, here is one way to do it.

In [ ]:
type(env)

In [ ]:
env.__dict__

In [ ]:
env._max_episode_steps = 2001

In [ ]:
env.__dict__